# Backtesting de Estrategia en XAUUSD

Resumen, lo hare cuando tengan las introudcciones de todos los modulos pero va algo asi:
Este notebook prepara los datos históricos de trading de XAUUSD para el backtesting de una estrategia basada en indicadores técnicos. A continuación, se detallan los pasos de carga, filtrado, cálculo de indicadores y limpieza de datos para una mejor precisión y control en los resultados del backtest.

---

## 1. Carga y Preparación de Datos

- ### Descarga de Datos
  Para obtener datos de alta calidad, descargué un total de 63 archivos CSV desde TradingView, con información de gráficos de 1 minuto para el par XAUUSD, abarcando el período del **4 de enero de 2021** al **25 de septiembre de 2024**.

- ### División del Conjunto de Datos
  Dividí los datos en dos subconjuntos para asegurar una evaluación equilibrada:

  - **Entrenamiento y Validación**: Datos desde el **4 de enero de 2021** hasta el **26 de abril de 2024**.
  - **Prueba**: Datos más recientes, desde el **29 de abril hasta el 25 de septiembre de 2024**.

- ### Límite de Horas de Operación
  Para centrarme en los periodos de mayor actividad y volatilidad, decidí limitar las horas de operación, mejorando así la precisión y relevancia de los resultados del backtest.

- ### Cálculo de Indicadores Técnicos
  Calculo una serie de indicadores técnicos necesarios para mi estrategia, asegurando que cada dato relevante esté disponible para el análisis.

- ### Verificación y Limpieza de Datos
  Realizo una limpieza exhaustiva de los datos, eliminando periodos con anomalías significativas, especialmente aquellos asociados a eventos de alto impacto (como noticias económicas) que podrían distorsionar el análisis del backtest.
isis del backtest.


In [1]:
import os
import pandas as pd
import pandas_ta as ta

# Función principal para cargar, filtrar, calcular indicadores y limpiar periodos
def preparar_datos(folder_path, start_date, end_date, periodos_a_eliminar):
    # Cargar todos los archivos CSV y concatenarlos en un DataFrame
    df_concatenado = pd.concat(
        [pd.read_csv(os.path.join(folder_path, f))
         for f in sorted(os.listdir(folder_path)) if f.endswith('.csv')],
        ignore_index=True
    )

    # Convertir la columna 'time' a formato datetime y ordenar
    df_concatenado['time'] = pd.to_datetime(df_concatenado['time'], unit='s')
    df_concatenado = df_concatenado.sort_values(by='time').reset_index(drop=True)

    # Filtrar por rango de fechas
    df_filtrado = df_concatenado[(df_concatenado['time'] >= start_date) & (df_concatenado['time'] <= end_date)]

    # Filtrar por horas de sesión en UTC (Londres y Nueva York)
    df_filtrado = df_filtrado.copy()  # Crear una copia explícita
    df_filtrado['hour'] = df_filtrado['time'].dt.hour  # Agregar la columna 'hour' sin advertencia
    mask_london = (df_filtrado['hour'] >= 5) & (df_filtrado['hour'] < 10)
    mask_newyork = (df_filtrado['hour'] >= 10) & (df_filtrado['hour'] < 16)
    df_filtrado = df_filtrado[mask_london | mask_newyork].drop(columns=['hour']).drop_duplicates(subset='time')

    # Calcular indicadores técnicos
    df_filtrado['ohlc_avg'] = (df_filtrado['open'] + df_filtrado['high'] + df_filtrado['low'] + df_filtrado['close']) / 4
    df_filtrado['EMA70'] = ta.ema(df_filtrado['ohlc_avg'], length=70)
    df_filtrado['EMA250'] = ta.ema(df_filtrado['ohlc_avg'], length=250)
    df_filtrado['RSI8'] = ta.rsi(df_filtrado['ohlc_avg'], length=8)

    # Eliminar filas nulas (primeros 250 valores debido a EMA250)
    df_filtrado = df_filtrado.iloc[250:].reset_index(drop=True)

    # Renombrar columnas para compatibilidad con librería de backtesting
    df_filtrado = df_filtrado.rename(columns={'open': 'Open', 'high': 'High', 'low': 'Low', 'close': 'Close'})
    df_filtrado.set_index('time', inplace=True)

    # Eliminar periodos específicos
    for inicio, fin in periodos_a_eliminar:
        df_filtrado = df_filtrado[~((df_filtrado.index >= inicio) & (df_filtrado.index <= fin))]

    return df_filtrado

# Ruta de la carpeta de los archivos CSV
folder_path = r"C:\Users\Roger Saavedra\Desktop\ML VS BACKTEST\Data\XAUUSD 2021-2024 M1"

# Definir los rangos de fechas y periodos a eliminar

# Entranamiento
start_date_train = pd.to_datetime('2021-01-04')
end_date_train = pd.to_datetime('2024-04-26')

# Prueba
start_date_test = pd.to_datetime('2024-04-29')
end_date_test = pd.to_datetime('2024-09-25')

# Valores anomalos identificados en el backtest
periodos_a_eliminar = [
    (pd.Timestamp('2022-04-19 15:55:00'), pd.Timestamp('2022-04-20 05:02:00')),
    (pd.Timestamp('2024-02-08 14:25:00'), pd.Timestamp('2024-02-08 14:45:00')),
    (pd.Timestamp('2024-01-18 13:24:00'), pd.Timestamp('2024-01-18 13:36:00')),
    (pd.Timestamp('2024-05-07 14:50:00'), pd.Timestamp('2024-05-07 15:10:00')),
    (pd.Timestamp('2024-07-05 12:20:00'), pd.Timestamp('2024-07-05 12:40:00'))
]

# Cargar, procesar y filtrar los datos de entrenamiento y prueba
dataf_entrenamiento = preparar_datos(folder_path, start_date_train, end_date_train, periodos_a_eliminar)

dataf_prueba = preparar_datos(folder_path, start_date_test, end_date_test, periodos_a_eliminar)

# Visualizacion del dataframe
dataf_prueba


,Open,High,Low,Close,Volume,ohlc_avg,EMA70,EMA250,RSI8
time,,,,,,,,,
2024-04-29 09:10:00,2339.380,2339.455,2339.030,2339.040,136,2339.22625,2337.780642,2334.036114,69.798570
2024-04-29 09:11:00,2339.040,2339.195,2338.985,2339.185,143,2339.10125,2337.817843,2334.076473,66.029232
2024-04-29 09:12:00,2339.185,2339.290,2339.035,2339.180,133,2339.17250,2337.856002,2334.117079,67.183682
2024-04-29 09:13:00,2339.180,2339.385,2338.300,2338.355,175,2338.80500,2337.882734,2334.154433,55.971263
2024-04-29 09:14:00,2338.355,2338.625,2338.270,2338.340,162,2338.39750,2337.897235,2334.188243,46.200204
...,...,...,...,...,...,...,...,...,...
2024-09-24 15:55:00,2640.490,2644.790,2639.905,2644.200,458,2642.34625,2642.983635,2638.054299,51.338494
2024-09-24 15:56:00,2644.190,2644.205,2642.165,2643.040,335,2643.40000,2642.995363,2638.096894,62.514041
2024-09-24 15:57:00,2643.045,2643.325,2642.245,2642.480,272,2642.77375,2642.989121,2638.134160,54.078550


## 2. Lógica de la Estrategia

La estrategia de trading utiliza dos criterios principales: la **detección de tendencia** y las **señales de compra y venta**.

- **Detección de Tendencia**:
   - Se considera que hay una tendencia **alcista** cuando la EMA de 70 periodos (rápida) está por encima de la EMA de 250 periodos (lenta).
   - De manera opuesta, se considera una tendencia **bajista** cuando la EMA de 70 periodos está por debajo de la EMA de 250 periodos.
     
- **Señales de Compra (Largos)**:
   - La estrategia busca un cruce del RSI por debajo del nivel 10, lo cual indica una situación de sobreventa.
   - Cuando el RSI cruza nuevamente hacia arriba del nivel 10, se ejecuta la operación de compra.
   - El **stop-loss (SL)** se coloca en el mínimo de las últimas 5 velas, menos un margen de 20 pips.
   - El **take-profit (TP)** se establece con un ratio riesgo/beneficio de 1:1.
     
- **Señales de Venta (Cortos)**:
   - Se detecta una señal de venta cuando el RSI cruza por encima del nivel 90, indicando sobrecompra.
   - Cuando el RSI vuelve a cruzar hacia abajo del nivel 90, se ejecuta la operación de venta.
   - El **SL** se coloca en el máximo de las últimas 5 velas, más un margen de 20 pips.
   - El **TP** se calcula con el mismo ratio de 1:1.
     
Esta lógica se implementa en el código utilizando la biblioteca `backtesting.py`, permitiendo realizar pruebas de la estrategia en ambos periodos de datos: el periodo de entrenamiento y el de prueba. Los resultados de estos backtests servirán como base para identificar mejoras en los parámetros y en la configuración de la estrategia.


#### Backtesting datos de entrenamiento 

In [2]:
from backtesting import Backtest, Strategy

# Definir el valor de un pip para XAUUSD
valor_pip = 0.01
margen_pips = 20  # Margen de 20 pips

# Función para detectar cruces hacia arriba (crossover)
def crossover(series, level):
    return series[-2] < level and series[-1] > level

# Función para detectar cruces hacia abajo (crossunder)
def crossunder(series, level):
    return series[-2] > level and series[-1] < level

# Función para validar si estamos en una tendencia alcista (largos)
def is_bullish_trend(ema_fast, ema_slow):
    return ema_fast > ema_slow

# Función para validar si estamos en una tendencia bajista (cortos)
def is_bearish_trend(ema_fast, ema_slow):
    return ema_fast < ema_slow

class EMARSIWithPipMarginStrategyCombined(Strategy):
    risk_reward_ratio = 1.0 # Riesgo beneficio 1:1 
    risk_amount = 100  # Riesgo fijo por operación

    def init(self):
        # Inicialización de las series del DataFrame 
        self.ema_fast = self.I(lambda: self.data['EMA70'])  # EMA rápida 
        self.ema_slow = self.I(lambda: self.data['EMA250'])  # EMA lenta 
        self.rsi = self.I(lambda: self.data['RSI8']) 
        
        # Variables para resetear señales
        self.rsi_below_10 = False  # Para manejar las señales de compra (largos)
        self.rsi_above_90 = False  # Para manejar las señales de venta (cortos)

    def is_within_no_trade_zone(self):
        # Obtener la hora actual del índice de datos
        current_time = self.data.index[-1]
        hour = current_time.hour
        minute = current_time.minute

        # Solo identificar si estamos en las últimas 30 velas (30 minutos) antes del cierre de la sesión de Nueva York
        if (hour == 15 and minute >= 30):  # 30 minutos antes de las 16:00 UTC (fin de sesión NY)
            return True
        return False

    def next(self):
        # Verificar si estamos dentro de la ventana de no operación
        if self.is_within_no_trade_zone():
            return  # No abrir operación

        # Verificar si ya hay una posición abierta
        if self.position:
            return  # No abrir una nueva operación si ya hay una posición abierta

        # Obtener los valores correspondientes de la barra actual
        ema_fast = self.ema_fast[-1]
        ema_slow = self.ema_slow[-1]

        ### Lógica de compra (largos)
        if is_bullish_trend(ema_fast, ema_slow):
            # Detectamos el cruce hacia abajo del nivel 10 del RSI (crossunder)
            if crossunder(self.rsi, 10) and not self.rsi_below_10:
                self.rsi_below_10 = True  # Marcamos que el RSI ha cruzado hacia abajo
            
            # Luego detectamos el cruce hacia arriba del nivel 10 (crossover) después de haber cruzado hacia abajo
            if self.rsi_below_10 and crossover(self.rsi, 10):
                sl = self.data.Low[-5:].min() - (margen_pips * valor_pip)
                tp = self.data.Close[-1] + (self.data.Close[-1] - sl) * self.risk_reward_ratio
                
                risk_per_unit = self.data.Close[-1] - sl
                
                if risk_per_unit > 0:
                    size = self.risk_amount / risk_per_unit
                    size = max(1, int(size))  # Aseguramos que el tamaño mínimo sea 1
                    self.buy(size=size, sl=sl, tp=tp)
                
                self.rsi_below_10 = False  # Reiniciamos la señal para el siguiente ciclo

        ### Lógica de venta (cortos)
        if is_bearish_trend(ema_fast, ema_slow):
            # Detectamos el cruce hacia arriba del nivel 90 del RSI (crossover)
            if crossover(self.rsi, 90) and not self.rsi_above_90:
                self.rsi_above_90 = True  # Marcamos que el RSI ha cruzado hacia arriba
            
            # Luego detectamos el cruce hacia abajo del nivel 90 (crossunder) después de haber cruzado hacia arriba
            if self.rsi_above_90 and crossunder(self.rsi, 90):
                sl = self.data.High[-5:].max() + (margen_pips * valor_pip)
                tp = self.data.Close[-1] - (sl - self.data.Close[-1]) * self.risk_reward_ratio
                
                risk_per_unit = sl - self.data.Close[-1]
                
                if risk_per_unit > 0:
                    size = self.risk_amount / risk_per_unit
                    size = max(1, int(size))  # Aseguramos que el tamaño mínimo sea 1
                    self.sell(size=size, sl=sl, tp=tp)
                
                self.rsi_above_90 = False  # Reiniciamos la señal para el siguiente ciclo

# Ejecutar el backtest con la estrategia combinada
bt_entrenamiento = Backtest(dataf_entrenamiento, EMARSIWithPipMarginStrategyCombined, cash=10000, margin=1/10000, commission=.000)
stats_entrenamiento = bt_entrenamiento.run()
print(stats_entrenamiento)


C:\Users\Roger Saavedra\anaconda3\Lib\site-packages\backtesting\_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

Start                     2021-01-04 09:10:00
End                       2024-04-25 15:59:00
Duration                   1207 days 06:49:00
Exposure Time [%]                    2.367802
Equity Final [$]                    11190.232
Equity Peak [$]                     13428.025
Return [%]                           11.90232
Buy & Hold Return [%]               20.570143
Return (Ann.) [%]                    2.776974
Volatility (Ann.) [%]               18.778143
Sharpe Ratio                         0.147883
Sortino Ratio                        0.219433
Calmar Ratio                         0.134353
Max. Drawdown [%]                  -20.669309
Avg. Drawdown [%]                   -1.972645
Max. Drawdown Duration      399 days 00:14:00
Avg. Drawdown Duration       18 days 04:28:00
# Trades                                 1458
Win Rate [%]                        50.411523
Best Trade [%]                       0.436483
Worst Trade [%]                     -0.344177
Avg. Trade [%]                    

### Disminuyo la cantidad de datos para poder hacer comprobaciones visuales de que la estrategia se esta ejecuntado de forma correcta

In [3]:
import warnings

# Ignorar todas las advertencias relacionadas con Bokeh y otras DeprecationWarnings
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)

# Filtrar las últimas 5000 velas del dataframe
filtered_data = dataf_entrenamiento.tail(5000)

# Ejecutar el backtest con los datos filtrados
bt_limited_entrenamiento = Backtest(filtered_data, EMARSIWithPipMarginStrategyCombined, cash=10000, margin=1/10000, commission=.000)
stats_limited_entrenamiento = bt_limited_entrenamiento.run()

# Graficar el backtest con las últimas 5000 velas
bt_limited_entrenamiento.plot()


GridPlot(id='p1371', ...)